<a href="https://colab.research.google.com/github/JuanM-GG/maestria/blob/main/ml-lote/dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install smt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 7.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for smt: filename=smt-1.3.0-cp38-cp38-linux_x86_64.whl size=541355 sha256=6ca6df0ae0625cdfcaf9c921a4795563fc4de37ab0c60a960f34e5a8d2dc5da2
  Stored in directory: /root/.cache/pip/wheels/20/30/9f/964979f39176b757a3a23584f6b7f00915d7595bc84f674297
  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-py3-none-any.whl size=25539 sha256=574e06344c7262814e3f23341e8df8905df297a96736cdd1e282dd1580d029a7
  Stored in directory: /root/.cache/pip/wheels/a7/de/26/98f9a9ff6082bdd6f2c003c6471f0fa7536b1ddd24dc2997e9
Successfully built smt pyDOE2


In [ ]:
pip install torchdyn

In [5]:
from typing import Tuple
import numpy as np
import torch

from enum import Enum
from smt.sampling_methods import LHS
from torchdyn.numerics import odeint

class Sampling(Enum):
    RANDOM = 0
    GRID = 1


def grid_init_samples(domain, n_trajectories: int) -> np.ndarray:
    x = np.linspace(domain[0][0], domain[0][1], n_trajectories)
    y = np.linspace(domain[1][0], domain[1][1], n_trajectories)

    xx, yy = np.meshgrid(x, y)
    return np.concatenate((xx.flatten()[..., np.newaxis], 
                           yy.flatten()[..., np.newaxis]), axis=1)


def random_init_samples(domain, n_trajectories: int) -> np.ndarray:
    values = LHS(xlimits=np.array(domain))
    return values(n_trajectories)


def pendulum(t, y):
    θ = y[:, 0]
    ω = y[:, 1]

    dθ = ω
    dω = -torch.sin(θ)

    return torch.stack((dθ, dω), dim=1)


def load_pendulum_data(t_span, 
                       y0s_domain=None,
                       n_trajectories=100, 
                       sampling=Sampling.RANDOM, 
                       solver='rk4') -> Tuple[torch.Tensor, torch.Tensor]:
    if not y0s_domain:
        y0s_domain = [[-1., 1.], [-1., 1.]]

    if sampling == Sampling.RANDOM:
        y0s = random_init_samples(y0s_domain, n_trajectories)
    elif sampling == Sampling.GRID:
        y0s = grid_init_samples(y0s_domain, n_trajectories)

    y0s = torch.tensor(y0s)
    _, ys = odeint(pendulum, y0s, t_span, solver) 

    return y0s, ys

In [15]:
grid_init_samples(((-1,1),(-1,1)), 3) # 3*3

array([[-1., -1.],
       [ 0., -1.],
       [ 1., -1.],
       [-1.,  0.],
       [ 0.,  0.],
       [ 1.,  0.],
       [-1.,  1.],
       [ 0.,  1.],
       [ 1.,  1.]])

In [20]:
random_init_samples(((-1,1),(-1,1)), 8)

array([[ 0.625,  0.625],
       [ 0.875, -0.125],
       [-0.875, -0.375],
       [ 0.375, -0.625],
       [-0.375,  0.375],
       [-0.125,  0.125],
       [ 0.125, -0.875],
       [-0.625,  0.875]])

In [39]:
domain = ((-10,-5),(15,20))
n_trajectories = 3
x = np.linspace(domain[0][0], domain[0][1], n_trajectories)
y = np.linspace(domain[1][0], domain[1][1], n_trajectories)

xx, yy = np.meshgrid(x, y)
print(xx,"\n\n", yy)

[[-10.   -7.5  -5. ]
 [-10.   -7.5  -5. ]
 [-10.   -7.5  -5. ]] 

 [[15.  15.  15. ]
 [17.5 17.5 17.5]
 [20.  20.  20. ]]


In [40]:
np.concatenate((xx.flatten()[..., np.newaxis], 
                yy.flatten()[..., np.newaxis]), axis=1)

array([[-10. ,  15. ],
       [ -7.5,  15. ],
       [ -5. ,  15. ],
       [-10. ,  17.5],
       [ -7.5,  17.5],
       [ -5. ,  17.5],
       [-10. ,  20. ],
       [ -7.5,  20. ],
       [ -5. ,  20. ]])

In [41]:
values = LHS(xlimits=np.array(domain))
values

In [42]:
n_trajectories = 3
values(n_trajectories)

array([[-9.16666667, 15.83333333],
       [-5.83333333, 17.5       ],
       [-7.5       , 19.16666667]])

In [54]:
y0s_domain = [[-1., 1-0.01], [0.01, 1.]]
n_trajectories = 3
_, y_pred = load_pendulum_data(torch.tensor([0,1]), 
                       y0s_domain=y0s_domain,
                       n_trajectories = n_trajectories, 
                       sampling=Sampling.RANDOM, 
                       solver='rk4')

print(_)
print()
print(y_pred)

tensor([[-0.6683,  0.8350],
        [-0.0050,  0.5050],
        [ 0.6583,  0.1750]], dtype=torch.float64)

tensor([[[-0.6683,  0.8350],
         [-0.0050,  0.5050],
         [ 0.6583,  0.1750]],

        [[ 0.3273,  1.0052],
         [ 0.4190,  0.2807],
         [ 0.5251, -0.4184]]], dtype=torch.float64)


In [55]:
print(y_pred.shape)

torch.Size([2, 3, 2])
